# **Desafio de Classificação:**

**Nome:** Gustavo Bartholomeu Trad Souza

**Nº USP:** 11219216

**Disciplina:** SCC0277 - Competições de Ciências de Dados

**Docente:** Fernando Pereira dos Santos

**Universidade de São Paulo (USP)**

Utilizando a metodologia básica de um projeto para ciência de dados, implemente possíveis soluções para o seguinte estudo de caso (https://www.kaggle.com/datasets/ahsan81/hotel-reservations-classification-dataset?resource=download). Desenvolva e implemente métodos que se propõem a responder a questão: **Será que um hóspede irá cancelar sua reserva?**

Em seu projeto deverá constar uma análise detalhada do dataset e descrição das transformações de dados realizadas com suas respectivas justificativas. Adicionalmente, crie um baseline com as técnicas contidas nesta aula. Como forma de superar este baseline, pesquise por métodos e técnicas mais complexas. Você deverá entregar um relatório com os seguintes itens:

1. Descrição do problema e análise dos dados
2. Descrição das técnicas utilizadas
3. Interpretação dos resultados obtidos
4. Conclusão
5. Apêndice (descrição das técnicas de Classificação apresentadas pelos colegas durante os seminários)

In [1]:
# Importação das bibliotecas

# pandas para a leitura e manipulação dos data frames
import pandas as pd

# matplotlib para o plot de gráficos
import matplotlib.pyplot as plt

# numpy para operações com arranjos
import numpy as np

## 1. Descrição do problema e análise dos dados

Os canais online de reserva de hotéis mudaram drasticamente as possibilidades de reserva e o comportamento dos clientes em relação à reserva de vagas. Um número significativo de reservas em hotéis é cancelado e os motivos típicos para cancelamentos incluem mudança de planos, conflitos de agendamento, etc. Uma forma de evitar grandes despesas por parte dos hotéis é tentar prever a possibilidade do cancelamento de acordo com informações fornecidas durante a reserva, dessa forma, quando um cliente 

O dataset é estruturado e é composto de 19 colunas com informações sobre reservas realizadas por clientes e algumas informações sobre o cliente que realizou a reserva. As colunas do dataset são:

* **Booking_ID:** Identificador único de cada reserva.
* **no_of_adults:** Quantidade de adultos.
* **no_of_children:** Quantidade de crianças.
* **no_of_weekend_nights:** Quantidade de noites de fim de semana (sábado ou domingo) que o hóspede reservou.
* **no_of_week_nights:** Quantidade de noites de dias de semana (segunda a sexta) que o hóspede reservou.
* **type_of_meal_plan:** Tipo de plano de refeição reservado pelo cliente.
* **required_car_parking_space:** Necessita de vaga no estacionamento? *(0-Não, 1-Sim)*.
* **room_type_reserved:** Tipo de quarto reservado pelo cliente. Os valores são cifrados (codificados) pela INN Hotels.
* **lead_time:** Quantidade de dias entre a data da reserva e a data de chegada.
* **arrival_year:** Ano da reserva.
* **arrival_month:** Mês da reserva.
* **arrival_date:** Dia do mês da reserva.
* **market_segment_type:** Designação do segmento de mercado.
* **repeated_guest:** O cliente ja se hospedou no hotel antes? *(0-Não, 1-Sim)*.
* **no_of_previous_cancellations:** Número de reservas anteriores que foram canceladas pelo cliente antes da reserva atual.
* **no_of_previous_bookings_not_canceled:** Número de reservas anteriores que não foram canceladas pelo cliente antes da reserva atual.
* **avg_price_per_room:** Preço médio por dia da reserva; os preços dos quartos são dinâmicos. *(em euros)*.
* **no_of_special_requests:** Número total de solicitações especiais feitas pelo cliente *(por exemplo, número do andar, vista do quarto, etc.)*.
* **booking_status:** Flag indicando se a reserva foi cancelada ou não.

In [2]:
data = pd.read_csv('hotel_reservations.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36275 entries, 0 to 36274
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Booking_ID                            36275 non-null  object 
 1   no_of_adults                          36275 non-null  int64  
 2   no_of_children                        36275 non-null  int64  
 3   no_of_weekend_nights                  36275 non-null  int64  
 4   no_of_week_nights                     36275 non-null  int64  
 5   type_of_meal_plan                     36275 non-null  object 
 6   required_car_parking_space            36275 non-null  int64  
 7   room_type_reserved                    36275 non-null  object 
 8   lead_time                             36275 non-null  int64  
 9   arrival_year                          36275 non-null  int64  
 10  arrival_month                         36275 non-null  int64  
 11  arrival_date   

### Atributos quantitativos e qualitativos

O dataset possui 36275 entradas distintas e **sem valores faltantes** com informações de reservas realizadas por clientes em diferentes hotéis. Dos 19 atributos presentes na tabela, 9 são quantitativos, sendo eles: **no_of_adults, no_of_children, no_of_weekend_nights, no_of_week_nights, lead_time, no_of_previous_cancellations, no_of_previous_bookings_not_canceled, avg_price_per_room, no_of_special_requests** e 9 atributos qualitativos, sendo eles: **type_of_meal_plan, required_car_parking_space, room_type_reserved, arrival_year, arrival_month, arrival_date, market_segment_type, repeated_guest, booking_status**. O atributo de identificação **Booking_ID** não é elevado em conta pois identifica univocadamente cada entrada da tabela e, portanto, não traz informações relevantes para a análise dos dados. Cada um dos atributo qualitativos possuem o seguinte número de valores únicos:

In [3]:
print('Numero de valores faltantes:')
data.isna().sum()

Numero de valores faltantes:


Booking_ID                              0
no_of_adults                            0
no_of_children                          0
no_of_weekend_nights                    0
no_of_week_nights                       0
type_of_meal_plan                       0
required_car_parking_space              0
room_type_reserved                      0
lead_time                               0
arrival_year                            0
arrival_month                           0
arrival_date                            0
market_segment_type                     0
repeated_guest                          0
no_of_previous_cancellations            0
no_of_previous_bookings_not_canceled    0
avg_price_per_room                      0
no_of_special_requests                  0
booking_status                          0
dtype: int64

In [4]:
# colunas quantitativas
quant = ['no_of_adults', 'no_of_children', 'no_of_weekend_nights', 'no_of_week_nights', 'lead_time', 'no_of_previous_cancellations', 
         'no_of_previous_bookings_not_canceled', 'avg_price_per_room', 'no_of_special_requests']

# colunas qualitativas
quali = ['type_of_meal_plan', 'required_car_parking_space', 'room_type_reserved', 'arrival_year', 'arrival_month', 'arrival_date', 
         'market_segment_type', 'repeated_guest']

target = ['booking_status']

print('Número de valores únicos nas colunas qualitativas:\n')
for col in quali:
    print(col, ':', data[col].nunique())
    print('\t', data[col].unique(), '\n')


Número de valores únicos nas colunas qualitativas:

type_of_meal_plan : 4
	 ['Meal Plan 1' 'Not Selected' 'Meal Plan 2' 'Meal Plan 3'] 

required_car_parking_space : 2
	 [0 1] 

room_type_reserved : 7
	 ['Room_Type 1' 'Room_Type 4' 'Room_Type 2' 'Room_Type 6' 'Room_Type 5'
 'Room_Type 7' 'Room_Type 3'] 

arrival_year : 2
	 [2017 2018] 

arrival_month : 12
	 [10 11  2  5  4  9 12  7  6  8  3  1] 

arrival_date : 31
	 [ 2  6 28 20 11 13 15 26 18 30  5 10  4 25 22 21 19 17  7  9 27  1 29 16
  3 24 14 31 23  8 12] 

market_segment_type : 5
	 ['Offline' 'Online' 'Corporate' 'Aviation' 'Complementary'] 

repeated_guest : 2
	 [0 1] 

booking_status : 2
	 ['Not_Canceled' 'Canceled'] 



### Correlação entre atributos quantitativos

In [5]:
corr = data[quant].corr()
corr

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,lead_time,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests
no_of_adults,1.000000,-0.019787,0.103316,0.105622,0.097287,-0.047426,-0.119166,0.296886,0.189401
no_of_children,-0.019787,1.000000,0.029478,0.024398,-0.047091,-0.016390,-0.021189,0.337728,0.124486
no_of_weekend_nights,0.103316,0.029478,1.000000,0.179577,0.046595,-0.020690,-0.026312,-0.004525,0.060593
no_of_week_nights,0.105622,0.024398,0.179577,1.000000,0.149650,-0.030080,-0.049344,0.022753,0.045994
lead_time,0.097287,-0.047091,0.046595,0.149650,1.000000,-0.045723,-0.078137,-0.062596,-0.101645
no_of_previous_cancellations,-0.047426,-0.016390,-0.020690,-0.030080,-0.045723,1.000000,0.468147,-0.063340,-0.003317
no_of_previous_bookings_not_canceled,-0.119166,-0.021189,-0.026312,-0.049344,-0.078137,0.468147,1.000000,-0.113684,0.027377
avg_price_per_room,0.296886,0.337728,-0.004525,0.022753,-0.062596,-0.063340,-0.113684,1.000000,0.184381
no_of_special_requests,0.189401,0.124486,0.060593,0.045994,-0.101645,-0.003317,0.027377,0.184381,1.000000


In [11]:

max_corr = 0.7

matrizSuperior = np.triu(np.ones(corr.shape),k=1).astype(bool)
correlacaoSuperior = corr.where(matrizSuperior)

atributos = [col for col in correlacaoSuperior.columns if any(abs(correlacaoSuperior[col]) >= max_corr)]
print("Atributos quantitativos com alta correlação:")
print(atributos)


Atributos quantitativos com alta correlação:
[]


Os atributos quantitativos possuem uma correlação muito baixa, sendo a maior correlação entre os atributos no_of_previous_cancellations e no_of_previous_bookings_not_canceled com o valor de 0.468147, portanto todos os valores serão mantidos 